# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [4]:
#TEST

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,karratha,-20.7377,116.8463,33.16,59,95,7.84,AU,1734404905
1,1,yellowknife,62.4560,-114.3525,-23.58,76,100,2.68,CA,1734404906
2,2,cabo san lucas,22.8909,-109.9124,22.09,82,0,0.00,MX,1734404907
3,3,hermanus,-34.4187,19.2345,14.33,71,42,3.32,ZA,1734404908
4,4,port-vila,-17.7338,168.3219,27.82,75,98,2.74,VU,1734404909


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
# Create the scatter plot without using geo=True
map_plot = city_data_df.hvplot.scatter(
    x="Lng",            # Longitude
    y="Lat",            # Latitude
    size="Humidity",    # Size of the points based on humidity
    color="City",       # Color of points based on city
    hover_cols=["City", "Humidity"],  # Information to display when hovering
    frame_width=650,    # Width of the plot
    frame_height=500,   # Height of the plot
    title="City Locations with Humidity Levels"
)

# Display the plot
map_plot

:Scatter   [Lng]   (Lat,City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_country_df = city_data_df.loc[city_data_df["Country"]=="AU"]
ideal_weather_max_df = ideal_country_df[ideal_country_df["Max Temp"] <= 25.00]
# ideal_weather_max_df = city_data_df[city_data_df["Max Temp"] <= 25.00]
ideal_city_df = ideal_weather_max_df[ideal_weather_max_df["Wind Speed"] >= 2.00]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,margaret river,-33.9500,115.0667,24.90,42,0,2.51,AU,1734404726
131,131,newnham,-41.3833,147.1333,20.53,42,20,10.80,AU,1734405057
240,240,mount gambier,-37.8333,140.7667,19.76,44,0,6.56,AU,1734405184
253,253,port lincoln,-34.7333,135.8667,20.14,53,0,7.27,AU,1734405200
343,343,codrington,-38.2667,141.9667,19.16,50,3,10.11,AU,1734405305


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = hotel_df.apply(lambda _: ' ', axis=1)


# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
5,margaret river,AU,-33.9500,115.0667,42,
131,newnham,AU,-41.3833,147.1333,42,
240,mount gambier,AU,-37.8333,140.7667,44,
253,port lincoln,AU,-34.7333,135.8667,53,
343,codrington,AU,-38.2667,141.9667,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
   "apiKey":geoapify_key,
   "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}" 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
newnham - nearest hotel: Riverside Hotel Motel
mount gambier - nearest hotel: The Old Mount Gambier Gaol
port lincoln - nearest hotel: Boston Hotel
codrington - nearest hotel: No hotel found
black point - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent


,City,Country,Lat,Lng,Humidity,Hotel Name
5,margaret river,AU,-33.9500,115.0667,42,Margaret River Hotel
131,newnham,AU,-41.3833,147.1333,42,Riverside Hotel Motel
240,mount gambier,AU,-37.8333,140.7667,44,The Old Mount Gambier Gaol
253,port lincoln,AU,-34.7333,135.8667,53,Boston Hotel
343,codrington,AU,-38.2667,141.9667,50,No hotel found
398,black point,AU,-34.6187,137.8972,47,No hotel found
492,new norfolk,AU,-42.7826,147.0587,61,Woodbridge on the Derwent


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
hotel_plot = hotel_df.hvplot.scatter(
    x='Lng',                 # Longitude
    y='Lat',                 # Latitude
    size='Humidity',         # Size based on humidity
    color='City',            # Color points by city
    hover_cols=["Hotel Name", "Country"],  # Hover message with additional information
    frame_width=650,         # Width of the plot
    frame_height=500,        # Height of the plot
    title="Hotel Locations with Humidity Levels"
)

# Display the plot
hotel_plot

:Scatter   [Lng]   (Lat,City,Humidity,Hotel Name,Country)